This tutorial will demonstrate how to:
- fit tuning curves given behavioral labels (e.g. position)
- perform state-space decoding in the fashion of [Denovellis et. al. (2021)](https://elifesciences.org/articles/64505). It gives the posterior probability of the *label* and the *dynamics type* (they call it continuous and discrete variable, repsectively). 
    - *dynamics type* specifies the temporal prior of the label. When the dynamics type is *continuous*, the temporal prior of the label is a gaussian random walk, with a movement variance specified by the user. When the dynamics type is *fragmented*, the temporal prior of the label is uniform across all possible bins.


# import  

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import poor_man_gplvm as pmg
import poor_man_gplvm.plot_helper as ph
import pynapple as nap

# Load the data (ignore this section and replace with your own data)
Some words on data preprocessing. We highly recommend [pynapple](https://pynapple.org/) as an entry point for neural data analysis in Python. They wrap around numpy objects but provide additional useful functionalities like restricting to time intervals, aligning to common time stamps, and turn spike times into counts. Essentially, for this tutorial, we need: 
- *spk_times*: pynapple TsGroup, obtained from a list of spike times (from the entire recording) for each unit.
- *position_tsdf*: pynapple TsdFrame, obtained from an array of (n_time, n_columns), timestamps, and column names. Each column is one behavior label we will decode (doesn't have to be position).
- *behavior_ep*: pynapple IntervalSet, obtained from arrays of start and end times of the behavior epoch when tuning curve is computed. 
- *speed_tsd* (optional): pynapple Tsd, obtained from an array of (n_time,) and timestamps. Here it is used for subselecting the locomotion epochs to include in the tuning curve computation. 

For the replay analysis later, we also need:
- *ep_full*: pynapple IntervalSet, for the whole recording where replay will be identified 
- *ep_nrem*: pynapple IntervalSet, for the non-REM episodes


In [11]:
import sys,os
sys.path.append('../../poor_gplvm/code')
import preprocess_roman_tmaze as preprt



data_dir_full = preprt.db_roman.iloc[0]['data_dir_full']

prep_res = nap.load_folder(os.path.join(data_dir_full, "derivatives"))  


spk_times = prep_res["spk_times"]
ripple_intervals = prep_res["ripple_intervals"]
position_tsdf = prep_res["position_tsdf"]
behavior_ep = prep_res['behavior_ep']


behavior_ep = prep_res['behavior_ep']
speed_tsd = prep_res['speed_tsd']

# epochs for replay analysis
ep_full = prep_res['full_ep']
ep_nrem = prep_res['sleep_state_intervals_NREMepisode']

# Prepare the data

## turn spike train into a matrix (TsdFrame, n_time x n_neuron) of spike counts 
Optional: use a mask to subselect only the pyramidal cells. This is easy if the relevant mask, e.g. *is_pyr* (whether it is a pyramidal cell) is stored as a metadata in the TsGroup.


In [4]:
spk_times_pyr=spk_times[spk_times['is_pyr']]
spk_mat = spk_times_pyr.count(0.1,ep=behavior_ep)

## prepare the labels and hyperparameters
### labels
In the paradigm of spatial navigation, the *label_l* can be time series of:
- linearized positions 
- 2D positions 
- [choice port ID](https://www.nature.com/articles/s41586-024-08397-7)
- linearized positions + direction
- 2D positions + direction

Indeed, in contrast to existing libraries of spatial decoding, we allow for arbitrary numbers of label dimension (up to memory constraint, so practically if one is already using 2D positions, the extra dimensions should not have too many discretized bins). 

Even for a linearizeable maze like the alternating T-maze, I personally still prefer using the 2D positions as labels. Whereas for a linear track, I would use the 1D position + direction, although 2D could give a subtler picture as hinted by [Zutshi et. al. (2025)](https://www.nature.com/articles/s41586-024-08397-7).


### Hyperparameters
- *label_bin_size*: binsize for discretizing the labels. 
- *smooth_std*: the standard deviation of the Gaussian kernel for smoothing the tuning curves. If None then no smoothing.
- *occupancy_threshold*: the occupancy threshold (in seconds) for the label bin to be considered valid. 

All of the above can be either: 1) a single number that apply to all the dimensions; 2) a list of value per dimension; and 3) for multiple mazes, a dictionary of {maze_key: val}, where val can be 1) or 2).

Here we will demonstrate the more general syntax assuming multiple mazes, but know that it can be simplified.

In [5]:
label_d= {} 
label_d['familiar']=position_tsdf[['x','y']].restrict(behavior_ep[0]) # The restrict limit the x y coordinates to the first behavior epoch

ep_d={}
ep_d['familiar'] = speed_tsd.restrict(behavior_ep[0]).threshold(5).time_support

label_bin_size_d = {}
label_bin_size_d['familiar'] =  3.

smooth_std_d = {}
smooth_std_d['familiar'] = 3.


## below is if there's a second novel linear maze, with time_window given by `behavior_ep[1]`

In [ ]:
# novel_lin=position_tsdf[['lin']].restrict(behavior_ep[1])

# novel_lin_dir = novel_lin.derivative() > 0

# beh_tsdf_novel=nap.TsdFrame(d=np.stack([novel_lin.d,novel_lin_dir.d.astype(int)],axis=1),t=novel_lin.t,columns=['lin','dir'])

# label_d['novel'] = beh_tsdf_novel

# speed_tsd_novel = np.abs(novel_lin.derivative())

# ep_d['novel'] = speed_tsd_novel.threshold(5).time_support



# label_bin_size_d['novel'] = [3.,1.]

# smooth_std_d['novel'] = [3,None]

# compute tuning curves
Tuning curves are computed by: 1) discretize the multi-dimensional labels into bins; 2) get occupancy of each bin; 3) drop the low occupancy bins; 4) compute the spike counts emitted within each bin; 5) smooth the occupancy and spike counts using a Gaussian kernel; 6) FR = count_smoothed / occupancy_smoothed, in Hz

In [6]:
from importlib import reload
reload(pmg)

<module 'poor_man_gplvm' from '/mnt/home/szheng/projects/poor-man-GPLVM/poor_man_gplvm/__init__.py'>

In [7]:
tuning_res = pmg.get_tuning_supervised(
    spk_mat,# nap.TsdFrame, n_time x n_neuron
    label_d,
    ep=ep_d,                   # nap.IntervalSet (optional)
    label_bin_size=label_bin_size_d,          # cm
    smooth_std=smooth_std_d,              # cm, Gaussian kernel std
    occupancy_threshold=0.1,     # seconds
)



[get_tuning] maze=familiar: grid_shape=(29, 28), n_neurons=422, dt=0.1000s, n_valid_time=8930, n_occupied_bins=306


# Replay analysis 

## preprocessing for replay analysis

### Identify population burst events
We bin spikes finely (`bin_size`,default 2ms), average across neurons (pyramidal cells), lightly smooth (`smooth_std`, default 7.5ms), z-score, identify the windows where the firing rate reachs beyond 3 standard deviations `z_thresh`, and extend both sides of the window back to the mean. The events are filtered by a `min_duration` and `max_duration`.

- `ep`: pynapple IntervalSet, time epochs where the population burst events are to be identified
- `threshold_ep`: pynapple IntervalSet, time epochs where the mean and standard deviation of firing rates are computed. Here we choose only the non-REM epochs.
- `force_reload`: whether to re-compute if the file already exists
- `dosave`: whether to save
- `return_population_rate`: whether to return the population firing rate, since it can be a long time series

Can optionally supply `ripple_intervals` (pynapple IntervalSet) so that the detected PBE is aware of how many sharp wave ripples are contained within. 

In [12]:
%%time
# get pbe
import poor_man_gplvm.post_fit_workflow.get_event_windows as gew

is_pyr = spk_times['is_pyr'] # assuming is_pyr is in the metadata of spk_times; otherwise a boolean array of length n_neuron would work
spk_times_pyr=spk_times[is_pyr] 
pbe_save_dir=os.path.join(data_dir_full,'py_data') # replace with the directory you like!
save_fn='pbe.p'#'pbe_wake_and_sleep_all.p'#


pbe_res=gew.detect_population_burst_event(spk_times_pyr, mask=None, ep=ep_full,threshold_ep=prep_res['sleep_state_intervals_NREMepisode'], bin_size=0.002, smooth_std=0.0075, 
                                 z_thresh=3.0, min_duration=0.05, max_duration=0.5,
                                 ripple_intervals=None,force_reload=False,dosave=False,save_dir=pbe_save_dir,return_population_rate=False,save_fn=save_fn)




/mnt/home/szheng/ceph/ad/roman_data/e13/e13_26m1/e13_26m1_210913/py_data/pbe.p exists; loading---
CPU times: user 97.1 ms, sys: 6.88 ms, total: 104 ms
Wall time: 150 ms


### Bin spikes
To work with data grouped by events / trials, we use a helper function that takes in `spk_times` (pynapple TsGroup) and `binsize` (default 0.02 s) and bin the spikes into two formats: `spike_mat_padded` and `spike_mat`.





In [15]:
import poor_man_gplvm.trial_analysis as tri
binsize = 0.02
spk_tensor_res=tri.bin_spike_train_to_trial_based(spk_times,pbe_res['event_windows'],binsize=binsize)

spk_tensor=spk_tensor_res['spike_mat_padded']
spk_mat_pbe=spk_tensor_res['spike_mat']
tensor_pad_mask=spk_tensor_res['mask']

In [20]:
print('Tensor format shape (n_event, n_time_max, n_neuron)',spk_tensor.shape)
print('Matrix format shape (n_time_concat, n_neuron)',spk_mat_pbe.shape)
print('Tensor pad mask shape (n_event, n_time_max, 1)',tensor_pad_mask.shape)


Tensor format shape (n_event, n_time_max, n_neuron) (5400, 25, 482)
Matrix format shape (n_time_concat, n_neuron) (30532, 482)
Tensor pad mask shape (n_event, n_time_max, 1) (5400, 25, 1)


# First question - whether an event is better-explained by the tuning templates than chance (goodness-of-fit, "on-manifold"ness)
For the first question we measure the model fit by the marginal likelihood of the Naive Bayes decoder. There is no temporal prior. It is asking whether the population vectors lie on the manifold of tuning curves, ignoring any question about temporal sequenceness. 

We use two shuffles to answer this question:
- *Neuron ID shuffle*: check whether the Neuron - tuning curve association is needed for the model fit
- *Circular shuffle*: check whether the **co-activity** is driving the model fit (rather than single neuron firing statistics)

Usually the *circular shuffle* test is harder to pass. 


In [ ]:
import poor_man_gplvm.post_fit_workflow.shuffle_test_decoding as shuf

dt = 0.02
gain = 1
flat_idx_to_coord = tuning_res['flat_idx_to_coord']
event_index_per_bin = spk_tensor_res['event_index_per_bin']
spk_mat_pbe = spk_tensor_res['spike_mat']

shuffle_res = shuf.shuffle_test_naive_bayes_marginal_l(
    spk_mat_pbe,
    event_index_per_bin,
    tuning=tuning_res['tuning_flat'],
    n_shuffle=100,
    sig_thresh=0.95,
    seed=0,
    decoding_kwargs={'dt': dt, 'gain': gain, 'n_time_per_chunk': 20000},
    dosave=True,
    force_reload=False,
    save_dir='./',
    save_fn='test_shuffle.p',
)